In [3]:
import pandas as pd
import os

In [5]:
# Files to load
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

In [6]:
school_data_df = pd.read_csv(school_data_to_load)
student_data_df = pd.read_csv(student_data_to_load)

In [23]:
# Initial exploration
print("Students dataframe dims:\n",student_data_df.count())
print("\nNulls in student dataframe:\n", student_data_df.isnull().sum())
print("\nData types in student dataframe:\n", student_data_df.dtypes)

print("\nSchools dataframe dims:\n",school_data_df.count())
print("\nNulls in schools dataframe:\n", school_data_df.isnull().sum())
print("Data types in schools dataframe:\n", school_data_df.dtypes)


Students dataframe dims:
 Student ID       39170
student_name     39170
gender           39170
grade            39170
school_name      39170
reading_score    39170
math_score       39170
dtype: int64

Nulls in student dataframe:
 Student ID       0
student_name     0
gender           0
grade            0
school_name      0
reading_score    0
math_score       0
dtype: int64

Data types in student dataframe:
 Student ID        int64
student_name     object
gender           object
grade            object
school_name      object
reading_score     int64
math_score        int64
dtype: object

Schools dataframe dims:
 School ID      15
school_name    15
type           15
size           15
budget         15
dtype: int64

Nulls in schools dataframe:
 School ID      0
school_name    0
type           0
size           0
budget         0
dtype: int64
Data types in schools dataframe:
 School ID       int64
school_name    object
type           object
size            int64
budget          int64
dtype:

In [26]:
school_data_df.budget.dtype

dtype('int64')